# Income Optimizer (Table-based)

Simple flow:
1. Load mock tables (`clients`, `client_employees`)
2. Compute user income from `client_employees.monthly_income`
3. Create optimization NBA events for users with income > threshold
4. Optionally post to backend endpoint

In [1]:
from nba_calculator.utils import CalculatorUtils, EnvConfig

In [2]:
# Configuration
ENDPOINT = EnvConfig.str("ENDPOINT", "http://127.0.0.1:8000/api/v1/internal/events/nba-calculation")
NBA_LIST_ENDPOINT = EnvConfig.str("NBA_LIST_ENDPOINT", "http://127.0.0.1:8000/api/v1/nba")
THRESHOLD = EnvConfig.float("THRESHOLD", 2000.0)
NBA_DEFINITION_ID = EnvConfig.str("NBA_DEFINITION_ID", "def_optimization")
SOURCE = EnvConfig.str("SOURCE", "calculator.income-threshold.v2")
REQUEST_ID_PREFIX = EnvConfig.str("REQUEST_ID_PREFIX", "calc-income")
DRY_RUN = EnvConfig.bool("DRY_RUN", False)  # Set false to POST events

In [3]:
def create_event_context(client, employee):
    context ={
            "monthly_income": employee.get("monthly_income"),
            "rule": f"monthly_income > {THRESHOLD}",
            "offer": "optimization",
            "client_id": client.get('client_id'),
            "employee_id": employee.get("employee_id")
            }
    return CalculatorUtils.set_context_hash(context)

def create_delete_event_context(client, employee):
    context = {
        "client_id": client.get('client_id'),
        "employee_id": employee.get("id"),
        "rule": f"monthly_income > {THRESHOLD}",
        "offer": "optimization",
        "message": f"This NBA is no longer applicable to you, since your income is below the threshold.",
        }
    
    return CalculatorUtils.set_context_hash(context)

In [4]:
# Get all employees that have an income above threshold
query = f"""
SELECT
    ce.id AS employee_id,
    ce.client_id,
    c.account_id,
    c.enterprise_number,
    ce.monthly_income
    FROM client_employees ce
  JOIN clients c ON c.id = ce.client_id
  WHERE ce.monthly_income > {THRESHOLD}
  ORDER BY ce.monthly_income DESC;
"""

client_employees_meeting_criteria = CalculatorUtils.execute_sql_query(query)

In [5]:
# Find clients employees that already have a linked, active optimization product
query = f"""
SELECT client_id, employee_id from client_products
where product_id = 'prd_optimization_basic' 
"""

client_employees_with_product = CalculatorUtils.execute_sql_query(query)

client_employee_mapping_set = dict()
for client in client_employees_with_product:
    client_id = client.get('client_id')
    employee_id = client.get('employee_id')
    
    tmp = client_employee_mapping_set.get(client_id,set())
    tmp.add(employee_id)
    
    client_employee_mapping_set[client_id] = tmp

In [6]:
clients_without_product = []

for client in client_employees_meeting_criteria:
    client_id = client.get('client_id')
    employee_id = client.get('employee_id')

    if employee_id not in client_employee_mapping_set.get(client_id, set()):
        clients_without_product.append(client)

In [7]:
new_events = []
for matching_client_employee in clients_without_product:
    client = { key: matching_client_employee.get(key) for key in ["account_id", "client_id", "enterprise_number"]}
    employee = matching_client_employee

    context = create_event_context(client, employee)
    event = CalculatorUtils.create_event(client, context,nba_definition_id=NBA_DEFINITION_ID, source=SOURCE)
    
    new_events.append(event)

In [8]:
CalculatorUtils.post_events(new_events, dry_run=DRY_RUN, endpoint=ENDPOINT, request_id_prefix=REQUEST_ID_PREFIX)

{'request_id': 'calc-income-1', 'event_id': 'c5016e30-71fa-4796-a377-6662395d696c', 'status': 202, 'response': '{"status":"accepted","event_id":"c5016e30-71fa-4796-a377-6662395d696c"}'}
{'request_id': 'calc-income-2', 'event_id': '05dddcc4-5f1d-498a-a117-51f2770054aa', 'status': 202, 'response': '{"status":"accepted","event_id":"05dddcc4-5f1d-498a-a117-51f2770054aa"}'}


In the [UI](http://localhost:8000/ui/active-nbas?client=9876543210) you should now see three active NBA's for client with enterprise_number "9876543210"